In [33]:
import googlemaps
import sqlite3
from datetime import datetime
import sys
import os
import importlib.util

# Absoluten Pfad zum gewünschten Ordner ermitteln
notebook_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "hidden"))

# Pfad zu sys.path hinzufügen
if notebook_dir not in sys.path:
    sys.path.append(notebook_dir)

# Modul dynamisch importieren
auth_path = os.path.join(notebook_dir, "auth.py")
spec = importlib.util.spec_from_file_location("auth", auth_path)
auth = importlib.util.module_from_spec(spec)
spec.loader.exec_module(auth)

from auth import google_auth # Personal Google API key is saved separately

In [34]:
conn = sqlite3.connect(r"..\data\processed\plz.sqlite")
cur = conn.cursor()

In [35]:
# Loads personalGgoogle API key and initializes googlemaps Client with it
auth_google = google_auth()
gmaps = googlemaps.Client(key=auth_google)

In [36]:
# cur.execute("""SELECT id, plz, lat, lng FROM Plz WHERE google_retrieved = 0""")
cur.execute(""" SELECT id, plz, lat, lng 
                FROM Plz 
                WHERE google_retrieved = 0 
                AND (plz LIKE '8%' OR plz LIKE '9%' OR plz LIKE '63%')""")
plzs = cur.fetchmany(1436)
# plzs = cur.fetchall()

In [37]:
len(plzs)

1435

In [38]:
# Sets the destinations all (clinica) and the depature time
destinations = ["LMU Klinikum Campus Großhadern Marchioninistraße 15, 81377 München, Germany",
                "Klinikum rechts der Isar der Technischen Universität München Ismaninger Str. 22, 81675 München, Germany",
                "Universitätsklinikum Würzburg Josef-Schneider-Straße 2, 97080 Würzburg, Germany",
                "Universitätsklinikum Erlangen Maximilianspl. 2, 91054 Erlangen, Germany",
                "Universitätsklinikum Regensburg Franz-Josef-Strauß-Allee 11, 93053 Regensburg, Germany",
                "Universitätsklinikum Augsburg Stenglinstraße 2, 86156 Augsburg, Germany"
                ]

departure_time = datetime(2025,3,13,7,0,0,0)
depature_time_str = departure_time.strftime("%Y/%m/%d, %H:%M")


In [39]:
print(depature_time_str)

2025/03/13, 07:00


In [40]:
# If no valid travel is returned by Google (ZERO_RESULTS) the value "None" is returned, else the travel distance value
def distance(results, plz, nr):
    if results["rows"][0]["elements"][nr]["status"] == "ZERO_RESULTS":
        print(plz, " without results for #", nr)
        return None
    elif results["rows"][0]["elements"][nr]["status"] == "OK":
        return results["rows"][0]["elements"][nr]["distance"]["value"]
    else:
        print("Unknown status:")
        print(results["rows"][0]["elements"][nr]["status"])

In [41]:
# If no valid travel is returned by Google (ZERO_RESULTS) the value "None" is returned, else the travel time value
def time(results, plz, nr):
    if results["rows"][0]["elements"][nr]["status"] == "ZERO_RESULTS":
        print(plz, " without results for #", nr)
        return None
    elif results["rows"][0]["elements"][nr]["status"] == "OK":
        return results["rows"][0]["elements"][nr]["duration"]["value"]
    else:
        print("Unknown status:")
        print(results["rows"][0]["elements"][nr]["status"])

In [42]:
# Loops over all retrieved communities and sends request to google maps distance_matrix (1 plz as origin and all clinica as destination)
# Gets results for travel by car and travel by transit.
# Parses the returned results (JSON Format) using functions "distance" and "time" (see above) and updates the database &sets google_retrieved = 1
k = 0
for plz in plzs:

    k += 1
    #origins = plz[1] + " " + plz[2] + ", Germany"
    origins = str(plz[2]) + "," + str(plz[3])
    print(k, ":",plz[1], origins)    

    results_car = gmaps.distance_matrix(origins=origins, destinations=destinations, departure_time=departure_time, mode="driving", traffic_model="best_guess")
    results_transit = gmaps.distance_matrix(origins=origins, destinations=destinations, departure_time=departure_time, mode="transit")
    
    car_distance_lmu = distance(results_car, plz, 0)
    car_time_lmu = time(results_car, plz, 0)
    car_distance_tum = distance(results_car, plz, 1)
    car_time_tum = time(results_car, plz, 1)
    car_distance_würzburg = distance(results_car, plz, 2)
    car_time_würzburg = time(results_car, plz, 2)
    car_distance_erlangen = distance(results_car, plz, 3)
    car_time_erlangen = time(results_car, plz, 3)
    car_distance_regensburg = distance(results_car, plz, 4)
    car_time_regensburg = time(results_car, plz, 4)
    car_distance_augsburg = distance(results_car, plz, 5)
    car_time_augsburg = time(results_car, plz, 5)

    transit_distance_lmu = distance(results_transit, plz, 0)
    transit_time_lmu = time(results_transit, plz, 0)
    transit_distance_tum = distance(results_transit, plz, 1)
    transit_time_tum = time(results_transit, plz, 1)
    transit_distance_würzburg = distance(results_transit, plz, 2)
    transit_time_würzburg = time(results_transit, plz, 2)
    transit_distance_erlangen = distance(results_transit, plz, 3)
    transit_time_erlangen = time(results_transit, plz, 3)
    transit_distance_regensburg = distance(results_transit, plz, 4)
    transit_time_regensburg = time(results_transit, plz, 4)
    transit_distance_augsburg = distance(results_transit, plz, 5)
    transit_time_augsburg = time(results_transit, plz, 5)

    cur.execute("""UPDATE Plz SET 
                        car_distance_lmu =?,             /*1*/
                        car_time_lmu =?,
                        car_distance_tum =?,
                        car_time_tum =?,
                        car_distance_würzburg =?,        /*5*/
                        car_time_würzburg =?,
                        car_distance_erlangen =?,
                        car_time_erlangen =?,
                        car_distance_regensburg =?,
                        car_time_regensburg =?,          /*10*/
                        car_distance_augsburg =?,
                        car_time_augsburg =?,
                        transit_distance_lmu =?,
                        transit_time_lmu =?,
                        transit_distance_tum =?,         /*15*/
                        transit_time_tum =?,
                        transit_distance_würzburg =?,
                        transit_time_würzburg =?,
                        transit_distance_erlangen =?,
                        transit_time_erlangen =?,        /*20*/
                        transit_distance_regensburg =?,
                        transit_time_regensburg =?,
                        transit_distance_augsburg =?,
                        transit_time_augsburg =?,
                        google_retrieved =1,             /*25*/
                        date_calc = ?
                    WHERE id=?""",
                    (   car_distance_lmu,                #1
                        car_time_lmu,
                        car_distance_tum,
                        car_time_tum,
                        car_distance_würzburg,           #5
                        car_time_würzburg,
                        car_distance_erlangen,
                        car_time_erlangen,
                        car_distance_regensburg,
                        car_time_regensburg,             #10
                        car_distance_augsburg,
                        car_time_augsburg,
                        transit_distance_lmu,
                        transit_time_lmu,
                        transit_distance_tum,            #15
                        transit_time_tum,
                        transit_distance_würzburg,
                        transit_time_würzburg,
                        transit_distance_erlangen,
                        transit_time_erlangen,           #20
                        transit_distance_regensburg,
                        transit_time_regensburg,
                        transit_distance_augsburg,
                        transit_time_augsburg,
                                                                        #25
                        depature_time_str,
                    plz[0]))         
    # print(origins)
conn.commit()                         



1 : 87776 48.000476,10.3640811
2 : 87778 48.01304289999999,10.4458036
3 : 87779 48.0019107,10.2159846
4 : 87781 48.00520239999999,10.2668505
(6867, '87781', 48.00520239999999, 10.2668505)  without results for # 0
(6867, '87781', 48.00520239999999, 10.2668505)  without results for # 0
(6867, '87781', 48.00520239999999, 10.2668505)  without results for # 1
(6867, '87781', 48.00520239999999, 10.2668505)  without results for # 1
(6867, '87781', 48.00520239999999, 10.2668505)  without results for # 2
(6867, '87781', 48.00520239999999, 10.2668505)  without results for # 2
(6867, '87781', 48.00520239999999, 10.2668505)  without results for # 3
(6867, '87781', 48.00520239999999, 10.2668505)  without results for # 3
(6867, '87781', 48.00520239999999, 10.2668505)  without results for # 4
(6867, '87781', 48.00520239999999, 10.2668505)  without results for # 4
(6867, '87781', 48.00520239999999, 10.2668505)  without results for # 5
(6867, '87781', 48.00520239999999, 10.2668505)  without results for

In [43]:
cur.close()
conn.close()